In [1]:
# Fixed Coupon Bond
# The goal of the project is to compute the market value of a portfolio of fixed coupon bonds taking into account the issuer 
# credit risk. The input parameters to derive this curve are:
# The par rate of a set of Overnight Index Swaps (i.e. their market quotation)
# A set of survival probabilities and the recovery rate of the issuer
# The static data of the pool of bonds (nominal, start date, end date, coupons, payment frequency)
# Notes and hints
# Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain 
# the theoretical framework for evaluation of risky flows.
# All maturities are expressed in months and rates are expressed as fractions of one (i.e. 0.01 means 1%).
# Remember to reuse the code we developed during the lessons in finmarkets.py as much as possible, instead of trying to rewrite
# everything from scratch!

In [2]:
ois_quotes = [
    {'maturity': 1, 'rate': 0.00106},
    {'maturity': 2, 'rate': 0.00114},
    {'maturity': 3, 'rate': 0.00115},
    {'maturity': 4, 'rate': 0.00117},
    {'maturity': 5, 'rate': 0.00119},
    {'maturity': 6, 'rate': 0.00121},
    {'maturity': 7, 'rate': 0.00122},
    {'maturity': 8, 'rate': 0.00124},
    {'maturity': 9, 'rate': 0.00128},
    {'maturity': 10, 'rate': 0.00131},
    {'maturity': 11, 'rate': 0.00135},
    {'maturity': 12, 'rate': 0.00138},
    {'maturity': 15, 'rate': 0.00152},
    {'maturity': 18, 'rate': 0.00166},
    {'maturity': 21, 'rate': 0.00184},
    {'maturity': 24, 'rate': 0.00206},
    {'maturity': 36, 'rate': 0.00344},
    {'maturity': 48, 'rate': 0.00543},
    {'maturity': 60, 'rate': 0.00756},
    {'maturity': 72, 'rate': 0.00967},
    {'maturity': 84, 'rate': 0.01162},
    {'maturity': 96, 'rate': 0.0134},
    {'maturity': 108, 'rate': 0.01502},
    {'maturity': 120, 'rate': 0.01649},
    {'maturity': 132, 'rate': 0.01776},
    {'maturity': 144, 'rate': 0.01888},
    {'maturity': 180, 'rate': 0.02137},
    {'maturity': 240, 'rate': 0.02322},
    {'maturity': 300, 'rate': 0.02389},
    {'maturity': 360, 'rate': 0.02416},
]

In [3]:
from finmarkets import DiscountCurve, OvernightIndexSwap, generate_swap_dates
from datetime import date
from dateutil.relativedelta import relativedelta
today = date(2013, 10, 31)
pillar_dates = [today]
swaps = []
for quote in ois_quotes:
    swap= OvernightIndexSwap (
        1e6,  
        generate_swap_dates (
            today,
            quote['maturity'],
        ),
         quote['rate']
    )
    
    swaps.append(swap)
    pillar_dates.append(swap.payment_dates[-1])

pillar_dates=sorted(pillar_dates)
n_df_vector=len(pillar_dates)



In [4]:
def objective_function(x):
    curve= DiscountCurve (
        today,
        pillar_dates,
        x   #discount factors
    )
    
    sum_sq =0.0
    for swap in swaps:
        sum_sq += swap.npv(curve) **2
    return sum_sq

In [5]:
from scipy.optimize import minimize

In [6]:
x0 = [1.0 for i in range(n_df_vector)]
bounds = [(0.01, 100.0) for i in range(n_df_vector)]

 
bounds[0] = (1.0, 1.0)

result = minimize(objective_function, x0, bounds=bounds)


In [7]:
result

      fun: 0.0006561289516799724
 hess_inv: <31x31 LbfgsInvHessProduct with dtype=float64>
      jac: array([  5.79535026e+05,  -2.83218934e-04,  -3.24325918e-04,
        -3.24159579e-04,  -3.11566549e-04,  -3.47202302e-04,
        -3.37286536e-04,  -2.87379278e-04,  -3.00610990e-04,
        -1.85489428e-04,  -1.04252349e-04,  -1.83492718e-04,
        -3.05678946e-03,   2.00371273e-03,   2.00847496e-03,
         1.81515215e-03,   5.34616839e-04,   2.40313017e-03,
         2.95535586e-03,   7.94011796e-04,  -1.57407167e-03,
        -3.01994456e-03,  -1.89445393e-03,  -4.25894649e-04,
         8.31555094e-04,   1.55296576e-03,   1.71100058e-03,
        -1.15041186e-03,   1.82715012e-03,   5.05944267e-04,
        -1.12756849e-03])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 448
      nit: 11
   status: 0
  success: True
        x: array([ 1.        ,  0.99991167,  0.99980687,  0.99970619,  0.99961015,
        0.99950111,  0.999392  ,  0.99928207,  0.99916713, 

In [8]:
curve = DiscountCurve(today, pillar_dates, result.x)
from datetime import date
curve.df(date(2016,8,4))

0.9910824292108902

In [9]:
 bonds_to_price = [
    {'nominal': 4972284.02, 'start_date': date(2010, 3, 1), 'end_date': date(2021, 8, 1),
     'coupon_frequency': 3, 'coupon': 0.035, 'recovery': 0.2},
    {'nominal': 7344328.27, 'start_date': date(2009, 7, 1), 'end_date': date(2022, 7, 1), 'coupon_frequency': 3, 'coupon': 0.035, 'recovery': 0.2},
    {'nominal': 7172290.19, 'start_date': date(2013, 1, 1), 'end_date': date(2017, 9, 1), 'coupon_frequency': 6, 'coupon': 0.02, 'recovery': 0.4},
    {'nominal': 7065224.23, 'start_date': date(2010, 3, 1), 'end_date': date(2014, 10, 1), 'coupon_frequency': 6, 'coupon': 0.02, 'recovery': 0.4},
    {'nominal': 5256452.14, 'start_date': date(2011, 7, 1), 'end_date': date(2016, 4, 1), 'coupon_frequency': 6, 'coupon': 0.02, 'recovery': 0.4},
    {'nominal': 2689680.89, 'start_date': date(2009, 9, 1), 'end_date': date(2024, 7, 1), 'coupon_frequency': 6, 'coupon': 0.02, 'recovery': 0.6},
    {'nominal': 3593518.71, 'start_date': date(2010, 7, 1), 'end_date': date(2019, 2, 1), 'coupon_frequency': 12, 'coupon': 0.02, 'recovery': 0.6},
    {'nominal': 6993589.53, 'start_date': date(2011, 1, 1), 'end_date': date(2018, 11, 1), 'coupon_frequency': 12, 'coupon': 0.02, 'recovery': 0.6},
    {'nominal': 6684377.52, 'start_date': date(2009, 9, 1), 'end_date': date(2021, 9, 1), 'coupon_frequency': 12, 'coupon': 0.02, 'recovery': 0.6},
    {'nominal': 6896199.04, 'start_date': date(2010, 7, 1), 'end_date': date(2018, 11, 1), 'coupon_frequency': 12, 'coupon': 0.027, 'recovery': 0.4},
    {'nominal': 2587984.6, 'start_date': date(2011, 10, 1), 'end_date': date(2020, 10, 1), 'coupon_frequency': 12, 'coupon': 0.02, 'recovery': 0.4},
    {'nominal': 3621656.1, 'start_date': date(2012, 6, 1), 'end_date': date(2016, 7, 1), 'coupon_frequency': 6, 'coupon': 0.027, 'recovery': 0.4},
    {'nominal': 3146567.47, 'start_date': date(2011, 6, 1), 'end_date': date(2022, 3, 1), 'coupon_frequency': 6, 'coupon': 0.018, 'recovery': 0.2},
    {'nominal': 6452721.61, 'start_date': date(2009, 4, 1), 'end_date': date(2019, 4, 1), 'coupon_frequency': 3, 'coupon': 0.018, 'recovery': 0.2},
    {'nominal': 3418346.24, 'start_date': date(2010, 5, 1), 'end_date': date(2016, 1, 1), 'coupon_frequency': 3, 'coupon': 0.018, 'recovery': 0.2},
]

In [10]:
from datetime import date
from dateutil.relativedelta import relativedelta

survival_probabilities = [
    {'date': date(2014, 12, 20), 'ndp': 0.972159727015014},
    {'date': date(2015, 12, 20), 'ndp': 0.942926329174406},
    {'date': date(2016, 12, 20), 'ndp': 0.913448056250137},
    {'date': date(2018, 12, 20), 'ndp': 0.855640452819766},
    {'date': date(2023, 12, 20), 'ndp': 0.732687779675469},
    {'date': date(2033, 12, 20), 'ndp': 0.539046016487758},
]




pillar_datess = [] # 2 "s" to distinguish from pillar dates of the Discount Curve
pillar_ndps = []

for quote in survival_probabilities:
    pillar_datess.append(quote ['date'])
    pillar_ndps.append(quote['ndp'])



In [11]:
from datetime import date
from dateutil.relativedelta import relativedelta
from math import log, exp
from finmarkets import DiscountCurve, CreditCurve
import numpy

def diff_month(d1, d2):
    return (d1.year - d2.year)*12 + d1.month - d2.month 
 

    
number_of_bonds = 0
     
mkt_value_portf = 0



for quote in bonds_to_price:


    N = quote['nominal']
    start_date = quote['start_date']
    end_date = quote['end_date']
    frequency = quote['coupon_frequency']
    coupon_rate = quote['coupon']
    recovery_rate = quote['recovery']
    lgd = 1- recovery_rate
       
     
    
    t_adj = diff_month(end_date, start_date)   # number of months between the start_date date and the end date
    kkk = t_adj % (12 / frequency)
    if kkk == 0:
        u = t_adj / (12 / frequency)
    else:
        u = (t_adj - 1) / (12 / frequency)
    
    periods = round(u)+1
    
    
 
    coupon = coupon_rate * N / frequency
     
    npv = 0
     
    diff = diff_month(end_date , start_date)
    xx = diff%(12/frequency)   # when xx in different from zero the Notional is given back one month later  
     
    
    days_between_payments = (12 / frequency) * 30    # how often the coupon is paid (in days)
     
    ccv = CreditCurve(pillar_datess, pillar_ndps)
    curve = DiscountCurve(today, pillar_dates, result.x)
    
    dist_today = (today - start_date).days
    ref_distance = (date(2014,12,20) - today).days
     
    # 2014/12/20  is the 1st pillar ndp date. Looking at the pillar ndps box it's clear that before 2014/12/20 the 
    # survival probability is 1 or quite close to 1 and thus the default probability for the period before that date
    # is negligible. Consequently the interpolation of ndps 
    # is done only for dates included between 2014/12/20 and 2033/12/20.

    
    for i in range(1, periods):
        days_between = round(days_between_payments * i)  # number of days between the start date and the single payment date
        bond_date = (start_date + relativedelta(days=days_between))
        bond_date_dist = (bond_date - start_date).days
        
        if bond_date_dist < dist_today:
            npv += 0
        else:
            right_distance = (bond_date - today).days
            disc_fact = curve.df(today + relativedelta(days = right_distance))
           
            if right_distance < ref_distance:
                npv += coupon * disc_fact
            else:
                
                ndp_yy = ccv.ndp(today + relativedelta(days = right_distance))
                hazard = ccv.hazard(today + relativedelta(days=right_distance))
                npv += coupon * disc_fact * exp(-(lgd * hazard* (right_distance/365)))
                
    # last coupon payment date
    lp = days_between_payments * periods 
    last_date = (start_date + relativedelta(days=lp))
    last_ndp = ccv.ndp(last_date)
    last_hazard = ccv.hazard(last_date)
    last_disc_fact = curve.df(last_date)
    last_distance = (last_date - today).days 
   
    
    # when the notional is given back one month later
    extra_lp = days_between_payments * (periods + 1)
    extra_date = (start_date + relativedelta(days=extra_lp))
    extra_ndp = ccv.ndp(extra_date)
    extra_hazard = ccv.hazard(extra_date)
    extra_disc_fact = curve.df(extra_date)
    extra_distance = (extra_date - today).days
     
    if xx == 0:     # the notional is given back on the same date of the last payment
        

         
        final = npv + coupon * last_disc_fact * exp(-(lgd * last_hazard * (last_distance/365)))  + N * last_disc_fact * exp(-(lgd * last_hazard * (last_distance/365))) 
         
    else:   # the notional is given back one month after the last coupon has been paid 

        final = npv + coupon * last_disc_fact * exp(-(lgd * last_hazard * (last_distance/365))) + N * extra_disc_fact * exp(-(lgd * extra_hazard * (extra_distance/365)))
    
    mkt_value_bond = round(final,1)
    
    
    number_of_bonds += 1 
    print  (  "    Mkt.val.Bond:  %s          " %   mkt_value_bond,"(", number_of_bonds, ")" )
    
    
    
        
     
    mkt_value_portf +=  mkt_value_bond
    
print("                                             Market value portf:  %s" % round( mkt_value_portf,1), "(", "Total" ,")")
    
        

    Mkt.val.Bond:  4841289.1           ( 1 )
    Mkt.val.Bond:  7150666.0           ( 2 )
    Mkt.val.Bond:  7049154.8           ( 3 )
    Mkt.val.Bond:  7195139.5           ( 4 )
    Mkt.val.Bond:  5221596.2           ( 5 )
    Mkt.val.Bond:  2464406.1           ( 6 )
    Mkt.val.Bond:  3594070.3           ( 7 )
    Mkt.val.Bond:  6990537.0           ( 8 )
    Mkt.val.Bond:  6453514.3           ( 9 )
    Mkt.val.Bond:  6918676.4           ( 10 )
    Mkt.val.Bond:  2433997.6           ( 11 )
    Mkt.val.Bond:  3668021.5           ( 12 )
    Mkt.val.Bond:  2678377.0           ( 13 )
    Mkt.val.Bond:  5977411.1           ( 14 )
    Mkt.val.Bond:  3353722.7           ( 15 )
                                             Market value portf:  75990579.6 ( Total )
